In [12]:
1#import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import scipy
from PIL import Image
from scipy import ndimage
#from lr_utils import load_dataset
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split#to load the dataset

In [4]:
#load dataset
#since we cant use lr_utils we use train_test_split from sklearn
#Also we need to load HDF5 dataset file, i think we can use pandas for the same
file_name=("train_catvnoncat.h5")
#f=h5py.File('train_catvnoncat.h5','r') as hdf:
#with h5py.File('train_catvnoncat.h5','r') as hdf:
    
 #   ls=list(hdf.keys())
#help(h5py.h5f.open)
#train_dataset = h5py.File('train_catvnoncat.h5', "r")
#pd.read_hdf('train_catvnoncat.h5')
sklearn.datasets.

SyntaxError: invalid syntax (<ipython-input-4-f5da797e279d>, line 12)

In [ ]:
import numpy as np
import h5py
    
    
def load_dataset():
    train_dataset = h5py.File('train_catvnoncat.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_catvnoncat.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [ ]:
train_set_x_orig,train_set_y,test_set_x_orig,test_set_y,classes=load_dataset()

In [14]:
#pwd
#load breast cancer dataset
cancer=load_breast_cancer()

In [16]:
cancer
#splitting to training and test sets
X_train,X_test,y_train,y_test=train_test_split(cancer.data,cancer.target,stratify=cancer.target,random_state=66)

In [21]:
X_train.shape

(426, 30)

In [24]:
#Construncting model()

#basic sigmoid

def sigmoid(z):
    """
    Compute the sigmoid of z
    Arguments:
    z -- A scalar or numpy array of any size"""
    
    s=1/(1+np.exp(-z))#finding the sigmoid
    return s

print("sigmoid([0,2])="+str(sigmoid(np.array([0,2]))))

sigmoid([0,2])=[0.5        0.88079708]


In [25]:
#implement parameter initialization, as vector of zeros

def initialize_with_zeros(dim):
    """this function creates a vector of zeros of shape (dim,1), for w and initialize b to 0
    
    Argument:
    dim---size of w vector we want
    (or number of parametersim this case)
    
    returns:
    w-- initialized vector of shape(dim,1)
    b--initialized scalar(corresponds to bias)"""
    #rows-dim, columns=1
    w=np.zeros((dim,1))
    b=0
    
    assert(w.shape==(dim,1))
    assert(isinstance(b,float)or isinstance(b,int))
    
    return w,b

In [8]:
dim=2
w,b=initialize_with_zeros(dim)
print("w="+str(w))
print("b="+str(b))

w=[[0.]
 [0.]]
b=0


In [57]:
#INITIALIZATION
w,b=initialize_with_zeros(X_train.shape[0]*X_train.shape[1])
X_train.shape[0]
#X_train[X_train.shape[0]*X_train.shape[1],1]
#X_train=X_train.reshape(X_train.shape[0]*X_train.shape[1],1)
#X_train.reshape(460,30)

12780

In [50]:
#FORWARD AND BACKWARD PROPAGATION

def propagate(w,b,X_train,y_train):
    """
    Implement the cost function and its gradient for the propagation 
    as in the equations explained
    
    Arguments:
    w--------weights, a numpy array of size (num_px*num_px*3,1)
    b---------bias scalar
    X---------data of size(num_px*num_px,number of examples)
    
    Y- the label vector(containing 0 if non-cat(malignant),1 if cat(benign)) of size(1,number of examples)
    
    Returns :
    cost------negative log likelihood cost for logistic regression
    dw--------gradient of the  loss with respect to w, same shape as w
    db--------gradient of the loss with respect to b, thus same shape as b
    
    Tips: write your code step by step for the propagation, use np.log, np.dot
    """
    m=X_train.shape[1]#assigning the size variable m
    
    #FORWARD PROPAGATION(FROM X TO COST)
    
    #COMPUTE ACTIVATION
    A=sigmoid(np.dot(w.T,X_train)+b)
    
    #Compute cost
    
    cost=(1/m)*(np.dot(y_train,np.log(A))+np.dot(1-y_train,np.log(1-A)))
    
    #gradient of loss with respect to w
    dw=(1/m)*np.dot(X,(A-y_train).T)
    #gradient of loss w.r.t. w
    db=(1/m)*(A-y_train)
    
    assert(dw.shape==w.shape)
    assert(db.dtype==float)
    cost=np.sqeeze(cost)
    assert(cost.shape==())
    
    grads={"dw":dw,"db":db}
    
    return grads,cost
    

In [56]:
#grads,cost=propagate(w,b,X_train,y_train)

In [1]:
def optimize(w,b,X_train,y_train,num_iterations,learning_rate,print_cost=False):
    """This function optimezes w and b by running a gradient descent algorithm
    
    Arguments:
    w------------weights, a numpy array of size (num_px*num_px*3,1)
    b -- bias, a scalar
    X -- data of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat), of shape (1, number of examples)
    num_iterations -- number of iterations of the optimization loop
    learning_rate -- learning rate of the gradient descent update rule
    print_cost -- True to print the loss every 100 steps
    
    Returns:
    params -- dictionary containing the weights w and bias b
    grads -- dictionary containing the gradients of the weights and bias with respect to the cost function
    costs -- list of all the costs computed during the optimization, this will be used to plot the learning curve.
    
    Tips:
    You basically need to write down two steps and iterate through them:
        1) Calculate the cost and the gradient for the current parameters. Use propagate().
        2) Update the parameters using gradient descent rule for w and b."""
    
    costs=[]
    for i in range(num_iterations):
        #cost and gradient calculation(~ 1-4 lines of code)
        grads,cost=propagate(w,b,X_train,y_train)
        
        #Retrive derivatives from  grads
        
        dw=grads["dw"]
        db=grads["db"]
        
        #update rule{~ 2 lines of code}
        #update the values with learning rate and gradient
        w=w-learning_rate*dw
        b=b-learning_rate*dw
        
        #Record the costs
        if i % 100==0:
            costs.append(cost)
            
        #print the cost every 100 training iterations
        
        if print_cost and i%100==0:
            print("cost after iteration % i: %f "%(i,cost))